### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# Importing libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [40]:
engine = create_engine('sqlite:///../DataBase.db') # nome do arquivo

df = pd.read_sql_table('merged_df',engine) # nome da tabela

x_input = 'message'

X = df[x_input]
Y = df.iloc[:,4:40]

In [3]:
df['genre'].unique()

array(['direct', 'social', 'news'], dtype=object)

### 2. Write a tokenization function to process your text data

In [4]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords  


nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'


def tokenize(text):

    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

        
    lemmatizer = WordNetLemmatizer()

    tokens = nltk.wordpunct_tokenize(text)
    text = nltk.Text(tokens)

    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in text if w.isalpha()]
    filtered_tokens = [w for w in clean_tokens if not w in stop_words]  


    return filtered_tokens

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Windows10\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Windows10\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
# message example
df.iloc[8,1]

'I would like to receive the messages, thank you'

In [6]:
tokenize(df.iloc[8,1])

['would', 'like', 'receive', 'message', 'thank']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [8]:

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
# y atre eache topic related
# X irs the message

In [10]:
%%time
pipeline.fit(X_train,y_train)

Wall time: 4.81 s


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000022947F16A60>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=KNeighborsClassifier()))])

In [11]:
%%time

y_pred = pipeline.predict(X_test)

Wall time: 2min 20s


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
from sklearn.metrics import classification_report


In [13]:
print(y_test.columns.to_list())

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [14]:
y_test

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
20984,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13888,1,0,0,1,0,0,0,0,0,0,...,0,0,1,1,0,0,1,0,0,0
24760,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23839,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1464,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11053,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
24008,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9612,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20890,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
y_pred

array([[2, 0, 0, ..., 0, 0, 0],
       [2, 0, 0, ..., 0, 0, 0],
       [2, 0, 0, ..., 0, 0, 0],
       ...,
       [2, 0, 0, ..., 0, 0, 0],
       [2, 0, 0, ..., 0, 0, 0],
       [2, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [35]:
target_names = y_test.columns.to_list()
for i,col in enumerate(y_test.columns):
    print(i,col)
    print(classification_report(y_test[col], y_pred[:,i]))

0 related
              precision    recall  f1-score   support

           0       0.66      0.13      0.22      1574
           1       0.84      0.08      0.14      4940
           2       0.01      0.93      0.01        40

    accuracy                           0.10      6554
   macro avg       0.50      0.38      0.12      6554
weighted avg       0.79      0.10      0.16      6554

1 request
              precision    recall  f1-score   support

           0       0.84      0.99      0.91      5437
           1       0.80      0.10      0.18      1117

    accuracy                           0.84      6554
   macro avg       0.82      0.55      0.55      6554
weighted avg       0.84      0.84      0.79      6554

2 offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6525
           1       0.00      0.00      0.00        29

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50   

In [17]:
len(y_pred[:,0])

6554

In [18]:
y_test[col]

20984    0
13888    0
24760    0
23839    0
1464     1
        ..
11053    0
24008    0
9612     0
20890    0
11129    0
Name: direct_report, Length: 6554, dtype: int64

### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
%%time

from sklearn.model_selection import GridSearchCV
pipeline.get_params()

Wall time: 997 µs


{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000022947F16A60>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=KNeighborsClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000022947F16A60>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=KNeighborsClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tf

In [23]:
%%time

# parameters = {
#     'vect__max_df': (0.5, 0.75, 1.0),
#     'vect__max_features': (None, 3000, 5000),
#     'vect__ngram_range': ((1,1),(1,2)) ,
#     'tfidf__use_idf': (True,False),
#     'clf__estimator__n_neighbors': (3,5,11,15),
#     'clf__estimator__weights': ('uniform','distance'),
#     'clf__estimator__metric': ('minkowski','euclidean','manhattan')

# }

parameters = {
    # 'vect__max_df': (0.5, 0.75),
    'vect__max_features': (1000, 3000),
    # 'vect__ngram_range': ((1,1),(1,2)) ,
    # 'tfidf__use_idf': (True,False),
    'clf__estimator__n_neighbors': (3,5),
    # 'clf__estimator__weights': ('uniform','distance'),
    # 'clf__estimator__metric': ('minkowski','euclidean')

}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=4)
cv_results = cv.fit(X_train,y_train)
print(cv.best_params_)

D:\Users\guilh\Anaconda3\envs\etl-env\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan]
  warnings.warn(
{'clf__estimator__n_neighbors': 3, 'vect__max_features': 1000}
Wall time: 5min 57s
Parser   : 272 ms


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [27]:
pipeline_tuned = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, max_features= 1000)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier(n_neighbors=3, metric='minkowski')))
])


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [31]:
import pickle
pipeline_tuned.fit(X_train,y_train)

filename = 'finalized_model.sav'
pickle.dump(pipeline_tuned, open(filename, 'wb'))
 
# some time later...
 

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [33]:
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.25328043942630457
